### Variables:
status --> whether ticket is sold/unsold \
ownership --> the owner of the specific ticket, useful to verify ownership later \
\
ticket_id --> a unique ID for each generated ticket which is used to check for duplicate tickets 




### mineBlock() Function - use of Nonce in a block
The mineBlock() function implements a proof of work algorithm. When we are creating a block, we initialize a variable in the Block class called 'nonce' - number only used once. 

This nonce is a part of the class variables, hence it is included in the block string. So, the hash generated is dependent on the 'nonce'. In the algorithm, we have some preset conditions set by the blockchain, which act as a **consensus mechanism**, which is the hash of the block starting with '0000' in our case. 

### Importing Libraries:
hashlib: for generating cryptographic hash for blocks\
datetime: for timestamping blocks\
time: for calculating time difference between block creation\
secrets, string: for generating secret keys\
hmac: for implementing HMAC verification\
json: for creating strings in a specific pattern

In [64]:
import hashlib
import hmac
import json
import datetime as dt
import time as time
import secrets
import string
import copy
import random

class Block:
    # class for creating blocks

    def __init__(self,index,timestamp,transactions,previous_hash,nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.transactions_data = ""
        for tr in self.transactions:
            i = tr["data"]
            self.transactions_data += i["ID"] + i["status"] + i["ownership"] 
        self.hash = self.HashCalculator()
    
    def HashCalculator(self): 
    # function to create hashes for an input of data

        transactions_info = str(self.index) + str(self.timestamp) + str(self.transactions_data) + self.previous_hash + str(self.nonce)
        string_data = json.dumps(transactions_info, sort_keys=True).encode()
        return hashlib.sha256(string_data).hexdigest() 
    
class BlockChain:
    # class for creating the blockchain

    def __init__(self):

        self.idx = 1 #indexing for added block
        self.chain = [] #starting chain with genesis block
        self.transactions_pool = [] #to store transactions yet to be included in the blockchain
        self.ledger = []
        self.event_details = [] #list of all the (date,time) tuples of the events whose tickets are created
        self.last_block_time = time.time()
        self.create_genesis_block()

    def create_genesis_block(self):
        #creating genesis block

        genesis_block = Block(0,dt.datetime.now(),[{"data":{"ID": "0", "status": "N/A", "ownership": "N/A"},\
                                                    "type":"genesis_creation"}],"0",0) 
        self.chain.append(genesis_block)

    def create_transaction(self,data,type,secret_key,buy_or_sell):
        #function to create a transaction

        challenge = generate_challenge()
        bit = bit_response()
        data_= json.dumps(data,sort_keys=True)
        data_for_hmac = data_ + challenge + str(bit)
        code_hmac = generate_HMAC(data_for_hmac,secret_key)

        return {"data": data, "challenge": challenge, "bit": bit, "type": type, "hmac": code_hmac, "buy/sell": buy_or_sell}

    def verifyTransaction(self,transaction,secret_key):
        #HMAC Verification for the transaction

        data_ = json.dumps(transaction["data"],sort_keys=True)
        data_for_verification = data_ + transaction["challenge"] + str(transaction["bit"])
        recalculate_hmac = generate_HMAC(data_for_verification,secret_key)

        if recalculate_hmac == transaction['hmac']:
            return True
        else:
            return False
        
    def check_create_block_criteria(self):
        #function to check if a block must be created

        if len(self.transactions_pool) >= 5 or (time.time() - self.last_block_time) >= 600:
            block = self.createBlock(dt.datetime.now(),self.transactions_pool)
            # self.mineBlock(block)
            bool_accept = self.accept_or_reject_block(block)

            if bool_accept:
                self.transactions_pool = [] #resetting transaction pool
                self.last_block_time = time.time()
        
    def add_transaction(self,transaction,secret_key):
        #function to add a new transaction to unconfirmed transactions

        if self.verifyTransaction(transaction,secret_key):
            copy_transaction = copy.deepcopy(transaction)
            self.transactions_pool.append(copy_transaction)
            self.ledger.append(transaction)
        
        if self.transactions_pool:
            self.check_create_block_criteria()
        
    def createBlock(self,timestamp,transactions):
        #function to create a block

        prev_hash = self.previous_block.hash 
        block = Block(self.idx,timestamp,transactions,prev_hash,nonce=0)
        return block  

    def mineBlock(self,block):
        #Proof of Work algorithm 

        block.nonce = 0
        hash = block.HashCalculator()
        while not hash.startswith('00'):
            block.nonce += 1
            hash = block.HashCalculator()
        return hash

    def validate_ProofOfWork(self,block):
        #Validation for the Proof of Work - A Consensus mechanism to check for eligibility 
        #to add the proposed block

        block.hash = self.mineBlock(block)
        return (block.hash.startswith('00') and block.previous_hash == self.chain[-1].hash)
    
    def accept_or_reject_block(self,block):
        #Accepting or rejecting blocks according to validation 

        if self.validate_ProofOfWork(block):
            self.chain.append(block)
            self.idx += 1 #updating index for next block  
            return True #to indicate block has been accepted
        
        return False
    
    @property
    def previous_block(self):
        return self.chain[-1]

def generate_challenge():
    #function to create a random string to be used in HMAC creation: Challenge-Response Authentication

    challenge = secrets.token_hex(16)
    return challenge

def bit_response():
    #function to generate a bit response to the challenge

    return random.randint(0,1)

def generate_HMAC(message,secret_key):
    #function which deals with HMAC generation

    return hmac.new(secret_key.encode(),message.encode(),hashlib.sha256).hexdigest()

#maintaining a user secret key database
user_key_database = {}

def generate_secret_key():
    #generating secret key for a user account

    length = 10
    options = string.ascii_letters + string.digits
    generated_key = ''.join(secrets.choice(options) for i in range(length))

    if generated_key not in user_key_database.values():
        return generated_key
    else:
        generate_secret_key()

user_key_database["organiser"] = generate_secret_key()
organiser_secret_key = user_key_database["organiser"]

blockchain = BlockChain()


''' CREATING TICKETS IN THE BLOCKCHAIN'''

def create_ticket(date,time,seat_no,secret_key):
    #function to create a ticket

    ticket_ID = date + time + seat_no 
    status = "unsold"
    ownership = "organiser"
    ticket_info = {
        "date": date,
        "time": time,
        "seat_number": seat_no,
        "status": status,
        "ownership": ownership,
        "ID": ticket_ID
    }

    transaction_create_ticket = blockchain.create_transaction(ticket_info,"create_ticket",secret_key,"None")

    blockchain.add_transaction(transaction_create_ticket,secret_key)


no_of_tickets_per_day = int(input("Enter number of tickets per day to be sold: "))
no_of_days = int(input("Enter number of days of event"))
total_no_of_tickets = no_of_tickets_per_day * no_of_days
# event_directory = {} 
for day_no in range(1, no_of_days+1):
    date = input(f"enter date for day {day_no} (Enter in DD-MM-YYYY format)")
    time_ = input(f"enter time for day {day_no} (Enter in 24-hr HH:MM format)")
    blockchain.event_details.append((date, time_, no_of_tickets_per_day))
    for i in range(1, no_of_tickets_per_day+1):
        seat_number = str(i)
        create_ticket(date,time_,seat_number,organiser_secret_key)


print("Number of Blocks created",blockchain.idx)
for i in blockchain.chain:
    print(i.__dict__)
        

Number of Blocks created 21
{'index': 0, 'timestamp': datetime.datetime(2024, 4, 21, 23, 4, 8, 695827), 'transactions': [{'data': {'ID': '0', 'status': 'N/A', 'ownership': 'N/A'}, 'type': 'genesis_creation'}], 'previous_hash': '0', 'nonce': 0, 'transactions_data': '0N/AN/A', 'hash': '1fa433767ac860a8521449fb0848ac43ba2543b4b85aa3048f06620e3bc95863'}
{'index': 1, 'timestamp': datetime.datetime(2024, 4, 21, 23, 4, 19, 695271), 'transactions': [{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '1', 'status': 'unsold', 'ownership': 'organiser', 'ID': '11-09-202418:001'}, 'challenge': '10b968765accd9fda9da5287061a7db3', 'bit': 1, 'type': 'create_ticket', 'hmac': 'af06193adfb5c32515291795ce777301813dfecf303d90b99f29d01af7d3baa5', 'buy/sell': 'None'}, {'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '2', 'status': 'unsold', 'ownership': 'organiser', 'ID': '11-09-202418:002'}, 'challenge': 'cd54f1db5edc589cb4d3a325fafe1422', 'bit': 0, 'type': 'create_ticket', 'hmac

In [80]:
'''CREATING USER ACCOUNTS'''

def get_userID(name,gender,age):
    #function to generate the user ID for the user
    
    user_ID = name + gender + age

    if user_ID not in user_key_database:
        return user_ID
    print("User Exists")
    raise KeyError

def create_user_account():
    #function to create a user account with his details

    name = input("Enter name")
    gender = input("Enter gender")
    age = input("Enter age")
    UserID = get_userID(name, gender, age)
    secretKey = generate_secret_key()
    user_key_database[UserID] = secretKey

    return UserID, secretKey

def ticket_status_check(ticket_id,blkchn,status,ownership):
    #function to check if the queried ticket is available for sale 

    for i in reversed(blkchn.ledger):
        k = copy.deepcopy(i["data"])
        if ticket_id == k["ID"]:
            #check status
            if k["status"] == status and k["ownership"] == ownership:
                return True,k
            return False,0
    print("Ticket is invalid")
    return False,1
            
def get_event_details(bkchn):
    #function to give event details to user

    print("All the dates, times, Total seats for the events are:",bkchn.event_details)


'''PURCHASE TICKETS'''

def purchase_ticket(user_id):
    #function to implement the ticket purchase from organiser process

    date = input("Which date is the event? (Enter in DD-MM-YYYY format)")
    time = input("What time is the event? (Enter in 24-hr HH:MM format)")
    seat_no = input("What seat number do you want?")
    
    ticket_id = date + time + seat_no
    available, ticket_inf = ticket_status_check(ticket_id,blockchain,"unsold","organiser")

    if available:
        ticket_inf["status"] = "sold"
        ticket_inf["ownership"] = user_id
        secretkey = user_key_database[user_id]
        transaction_purchase = blockchain.create_transaction(ticket_inf,"purchase_ticket",secretkey,user_id)
        blockchain.add_transaction(transaction_purchase,secretkey)
    else:
        if ticket_inf == 0:
            print("Ticket is not available")
        purchase_ticket(user_id)

'''SELL TICKETS'''

def sell_ticket(user_id,ticket_id):
    #function to implement the ticket sale to organiser process
    
    valid_owner, ticket_inf = ticket_status_check(ticket_id,blockchain,"sold",user_id)

    if valid_owner:
        ticket_inf["status"] = "unsold"
        ticket_inf["ownership"] = "organiser"
        secretkey = user_key_database[user_id]
        transaction_sell = blockchain.create_transaction(ticket_inf,"sell_ticket",secretkey,user_id)
        blockchain.add_transaction(transaction_sell,secretkey)
    else:
        if ticket_inf == 0:
            print("Ticket doesn't belong to you")
        

'''VERIFY USER'S TICKET AT ENTRY'''

def verify_user_ticket(user_id,ticket_id):
    #function to verify user's ticket at entry

    valid_owner, ticket_inf = ticket_status_check(ticket_id,blockchain,"sold",user_id)

    return valid_owner

In [66]:
# Person A - Create Account 
id, key = create_user_account()
print(id,key)

ABCMale50 SJ0rFFTuvT


In [69]:
# Person A - Purchase
get_event_details(blockchain)
purchase_ticket(id)
print(blockchain.ledger[-1])

All the dates, times, Total seats for the events are: [('11-09-2024', '18:00', 50), ('12-09-2024', '19:00', 50)]
{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '26', 'status': 'sold', 'ownership': 'ABCMale50', 'ID': '11-09-202418:0026'}, 'challenge': 'b19d163e2ac0571194fca99c1657aa6d', 'bit': 1, 'type': 'purchase_ticket', 'hmac': '0e049bfb98a7a23ac17bde95d10334a325749421ab75527db131c354d5acdef8', 'buy/sell': 'ABCMale50'}


In [70]:
# Person A - Sell
ticket = input("enter your ticketID")
sell_ticket(id,ticket)

print(blockchain.ledger[-1])

{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '26', 'status': 'unsold', 'ownership': 'organiser', 'ID': '11-09-202418:0026'}, 'challenge': '2561278dda57de96a560931033e16128', 'bit': 1, 'type': 'sell_ticket', 'hmac': '91c9a31a74111d8158aca21ab15c7d342bfab056b350ef330ea00e4e501c4c37', 'buy/sell': 'ABCMale50'}


In [71]:
# Person B - Create and Purchase the resold ticket
id2, key2 = create_user_account()
get_event_details(blockchain)
purchase_ticket(id2)
print(blockchain.ledger[-1])

All the dates, times, Total seats for the events are: [('11-09-2024', '18:00', 50), ('12-09-2024', '19:00', 50)]
{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '26', 'status': 'sold', 'ownership': 'DEFFemale45', 'ID': '11-09-202418:0026'}, 'challenge': '535f88196606822ae090ae152b79b84f', 'bit': 0, 'type': 'purchase_ticket', 'hmac': 'e9b33d01e630522bf2d7957440042fba4d7862c0d53da5c3ed9b3dbae894dbe6', 'buy/sell': 'DEFFemale45'}


In [72]:
'''Viewing list of transactions for a user'''

def viewUser(blkchn,userid):
    for i in blkchn.ledger:
        if i["buy/sell"] == userid:
            print(i)

In [73]:
# Printing person A transactions
id_view = input("Enter User ID")
viewUser(blockchain,id_view)

# # Printing person B transactions
# viewUser(blockchain,"BMale56")

{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '26', 'status': 'sold', 'ownership': 'ABCMale50', 'ID': '11-09-202418:0026'}, 'challenge': 'b19d163e2ac0571194fca99c1657aa6d', 'bit': 1, 'type': 'purchase_ticket', 'hmac': '0e049bfb98a7a23ac17bde95d10334a325749421ab75527db131c354d5acdef8', 'buy/sell': 'ABCMale50'}
{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '26', 'status': 'unsold', 'ownership': 'organiser', 'ID': '11-09-202418:0026'}, 'challenge': '2561278dda57de96a560931033e16128', 'bit': 1, 'type': 'sell_ticket', 'hmac': '91c9a31a74111d8158aca21ab15c7d342bfab056b350ef330ea00e4e501c4c37', 'buy/sell': 'ABCMale50'}


In [81]:
# Checking result of asking for an invalid ticket
id3, key3 = create_user_account()
get_event_details(blockchain)
purchase_ticket(id3)
print(blockchain.ledger[-1])

All the dates, times, Total seats for the events are: [('11-09-2024', '18:00', 50), ('12-09-2024', '19:00', 50)]
Ticket is invalid
{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '49', 'status': 'sold', 'ownership': 'HBKJMale23', 'ID': '11-09-202418:0049'}, 'challenge': 'c234afcc1622eff2474d3f52add31f04', 'bit': 0, 'type': 'purchase_ticket', 'hmac': '9b5da7d8d4f4e5225c47cfae7c160123a5e8c379089ba7b75fef24b2b3fe9ac2', 'buy/sell': 'HBKJMale23'}


In [74]:
print("Number of Blocks created",blockchain.idx)
for i in blockchain.chain:
    print(i.__dict__)

Number of Blocks created 21
{'index': 0, 'timestamp': datetime.datetime(2024, 4, 21, 23, 4, 8, 695827), 'transactions': [{'data': {'ID': '0', 'status': 'N/A', 'ownership': 'N/A'}, 'type': 'genesis_creation'}], 'previous_hash': '0', 'nonce': 0, 'transactions_data': '0N/AN/A', 'hash': '1fa433767ac860a8521449fb0848ac43ba2543b4b85aa3048f06620e3bc95863'}
{'index': 1, 'timestamp': datetime.datetime(2024, 4, 21, 23, 4, 19, 695271), 'transactions': [{'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '1', 'status': 'unsold', 'ownership': 'organiser', 'ID': '11-09-202418:001'}, 'challenge': '10b968765accd9fda9da5287061a7db3', 'bit': 1, 'type': 'create_ticket', 'hmac': 'af06193adfb5c32515291795ce777301813dfecf303d90b99f29d01af7d3baa5', 'buy/sell': 'None'}, {'data': {'date': '11-09-2024', 'time': '18:00', 'seat_number': '2', 'status': 'unsold', 'ownership': 'organiser', 'ID': '11-09-202418:002'}, 'challenge': 'cd54f1db5edc589cb4d3a325fafe1422', 'bit': 0, 'type': 'create_ticket', 'hmac